Author: Dennis Trollsfjord

Followed tutorial by tensorflow found [here](https://www.tensorflow.org/tutorials/images/classification), with modifications adapted for our implementation. 

In [98]:
%%capture
pip install matplotlib numpy tensorflow tensorflow_datasets --user

In [99]:
import matplotlib.pyplot as plt
import numpy as np
# import PIL
import tensorflow as tf

# from tensorflow import keras
# from tensorflow.keras import layers
# from tensorflow.keras.models import Sequential

import tensorflow_datasets as tfds  


[Tensorflow datasets](https://www.tensorflow.org/datasets/overview)

[Dataset](https://www.tensorflow.org/datasets/catalog/rock_paper_scissors)

In [100]:
# Load dataset whole dataset.
dataset, dataset_info = tfds.load(
    name='rock_paper_scissors', 
    split=["test", "train[0%:20%]", "train[20%:]"],
    data_dir='dataset', 
    with_info=True,
    as_supervised=True
)

assert isinstance(dataset, list) 
assert isinstance(dataset_info, tfds.core.dataset_info.DatasetInfo)

dataset_test, dataset_validate, dataset_train = dataset
dataset_label = dataset_info.features['label'].names
dataset_label_size = dataset_info.features['label'].num_classes
dataset_train_size = dataset_info.splits['train[20%:]'].num_examples
dataset_validate_size = dataset_info.splits['train[0%:20%]'].num_examples
dataset_test_size = dataset_info.splits['test'].num_examples

print(f"dataset name: {dataset_info.name}")  
print(f"dataset size: {dataset_info.dataset_size}")
print(f"dataset labels: {dataset_label}")
print(f"number of labels: {dataset_label_size}")
print(f"train dataset size: {dataset_train_size}") 
print(f"validate dataset size: {dataset_validate_size}") 
print(f"test dataset size: {dataset_test_size}")  

dataset name: rock_paper_scissors
dataset size: 219.23 MiB
dataset labels: ['rock', 'paper', 'scissors']
number of labels: 3
train dataset size: 2016
validate dataset size: 504
test dataset size: 372


In [101]:
def _plot_image(n=1) -> None:
    """Plots a image and associated label."""
    for image, label in dataset_train.take(n):
        plt.imshow(image.numpy())
        plt.title(dataset_label[label.numpy()])

In [102]:
batch_size = 10
image_size = {
    "height" : 64,
    "width"  : 64
}

In [103]:
AUTOTUNE = tf.data.AUTOTUNE

dataset_train = dataset_train.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
dataset_validate = dataset_validate.cache().prefetch(buffer_size=AUTOTUNE)

In [104]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip(
        mode="horizontal",
        input_shape=(image_size["height"], image_size["width"], 3),
    ),
    tf.keras.layers.RandomRotation(factor=0.1),
    tf.keras.layers.RandomZoom(height_factor=0.1, width_factor=0.1)
])

In [105]:
model = tf.keras.Sequential([
    data_augmentation,
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(dataset_label_size, name="outputs")
])

In [106]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_5 (Sequential)   (None, 64, 64, 3)         0         
                                                                 
 rescaling_2 (Rescaling)     (None, 64, 64, 3)         0         
                                                                 
 conv2d_6 (Conv2D)           (None, 64, 64, 16)        448       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 32, 32, 16)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 32, 32, 32)        4640      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                  

In [110]:
epochs = 10
history = model.fit(
    x=dataset_train,
    validation_data=dataset_validate,
    steps_per_epoch=dataset_train_size // epochs,
    validation_steps=dataset_validate_size // epochs,
)

ValueError: in user code:

    File "c:\Python310\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Python310\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Python310\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Python310\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "c:\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Python310\lib\site-packages\keras\engine\input_spec.py", line 295, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 of layer "sequential_6" is incompatible with the layer: expected shape=(N